In [14]:
# load libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import wrds
import pandas_datareader.data as web

In [15]:
# define functions  
def get_daily_wrds(conn,tickers=None):
  
    # Retrieve PERMNOs for the specified tickers
    permnos = conn.get_table(library='crsp', table='stocknames', columns=['permno', 'ticker', 'namedt', 'nameenddt'])
    permnos['nameenddt']=pd.to_datetime(permnos['nameenddt'])
    permnos = permnos[(permnos['ticker'].isin(tickers)) & (permnos['nameenddt']==permnos['nameenddt'].max())]
    # Extract unique PERMNOs
    permno_list = permnos['permno'].unique().tolist()
    print(permno_list)

    # Query daily stock file for the specified PERMNOs
    
    query = f"""
        SELECT permno, date, ret, retx, prc       
        FROM crsp.dsf
        WHERE permno IN ({','.join(map(str, permno_list))})
        ORDER BY date,permno
    """
    daily_returns = conn.raw_sql(query, date_cols=['date'])
    daily_returns = daily_returns.merge(permnos[['permno', 'ticker']], on='permno', how='left')
# Pivot data to have dates as index and tickers as columns
    daily_returns = daily_returns.pivot(index='date', columns='ticker', values='ret').dropna()    
    daily_returns=daily_returns[tickers]

    return daily_returns


def get_factors(ff6=False):
    
    fama_french = web.DataReader("F-F_Research_Data_5_Factors_2x3_daily", "famafrench",start="1926-01-01")
    daily_data = fama_french[0]  # The first DataFrame contains the daily data
    daily_data.index = pd.to_datetime(daily_data.index)  # Ensure index is datetime
    if ff6==False:
        df_factor = daily_data[['RF','Mkt-RF']]  # Extract the Risk-Free Rate (RF)
    else:
        df_factor = daily_data[['RF','Mkt-RF','SMB','HML','RMW','CMA']]
        fama_french = web.DataReader("F-F_Momentum_Factor_daily", "famafrench",start="1926-01-01")
        df_factor=df_factor.merge(fama_french[0],on='Date')
    
    return df_factor/100

# Introduction to Factor models 

Let $r^e$ be an asset excess return over the risk-free rate

Let $f^e$ be a traded factor, for now  the excess returns on SPY ETF

Then we can write

$$r^e=\alpha + b*f+\epsilon$$

where $b$ is asset $i$ exposure to the factor $f$

We can always write things that way--it is just a statistical decomposition

1. It can be useful for measuring risk if
    - The risk $\epsilon$ is specific to asset $i$ and uncorrelated with other assets
    - I.e the factors picks up lots of the variation in stock returns 

2. It can be useful for measuring expected returns if
    - The premium on the factor f, $E[f]$ summarize well the known "priced" systematic risk factors
    - By "priced" we mean risk-factors that earn a risk premium  $E[f]>0$

We will call the first a factor model of risk, and the second a factor model of expected returns


What use is a factor model?

- the alpha/Beta decomposition is how the money management industry is organized. 
    - Big pay days only if people think you have alpha
    - Getting exposure to the factor is a commodity. Anyone can buy SPY or many other funds that track the market very well
    - In most funds, specially for portfolio managers working a pod shops like Citadel and Millenium, a portfolio manager mandate is to take no factor risk--or at least aim to be close to factor neutral.
    - It is all about the alpha 
- factor models allow you to understand and hedge risk
    - This is essential if you want to concentrate your portfolio only on the pieces that you get compensated


## Alpha and Beta(s)

Note that

$$E[r]=E[rf+r^{e}]=rf+E[\alpha_i + b*f+\epsilon]$$

$$E[r]=rf+\alpha+b*E[f]+E[\epsilon]$$

- Using that $E[\epsilon]=0$ we have
$$E[r]=rf+\alpha+b*E[f]$$

- Thus, just like the risk of our asset could be attributed to the factor and it's component, the same model also allow us to decompose the expected return in terms of the risk-free rate, the premium coming from the common factor exposure $b*E[f]$ and the risk-premium specific to the asset, the ALPHA.


- The alpha does not need to be in a specific asset, can be o a combination of them

- The decomposition works the same for portfolio of assets

- For now we have a single factor, but you will see that as you have more factors, everything is pretty much the same



## Focus on your edge, hedge all the rest

No matter who you are, you will face dollar risk limits

So it is essential that you deploy risk wisely

The key use of the factor model is to give you discipline to separate your edge from the rest

How to use the factor model?

First , realize that each piece of the factor model will require very thoughtful estimation

Even here with only one asset and one factor we already have to know many things

1. The risk-free rate
2. The beta 
3. The factor risk-premia
4. The factor variance
5. The asset specific alpha
6. The factor specific variance
   
For now we will assume we know each of these.


if you have an asset that has alpha, says, you are confident that APPLE will beat expectations and you expect a 5% appreciation in the next 12 months beyond any factor exposure that APPle might had, what do you do?

1. Buy apple! 

But what is the market crashes? Why bear that risk?  
Portfolio allocators look for managers with alpha, not beta. 
What if the market crashes before apple surprises and your investors pull out?


2. Hedge the market
   - How do you do that?
   - Short it!
   - How much?
   - the asset beta!


Example:

1. You buy 1M in Apple stock, so your portfolio PnL is 1M*$r^{apple}$
2. Your portfolio PnL in excess what you would earn in treasuries is 1M*$(r^{apple}-rf)$
3. Sell (short) 1M*$beta_{apple,f}$ of the factor
4. Now your portfolio is PnL above the risk-free rate is

$$1M*(r^{apple}-rf-\beta*r^{SPY}+\beta*rf)$$

When you sell SPY short you get the money and here I am "investing" it in the risk-free rate

Alternatively you can think of it as using to fund the apple buy

it does not matter, because we are accounting for the time-value of money

We certainly don't want to invest anything else to just add noise to our trade 


$$1M*(r^{apple}-\beta*r^{SPY}-(1-\beta)*rf)$$

For a high beta stock your trade generates extra cash!

If $\beta>1$, $-(1-\beta)*rf>0$

This happens because you are selling more than you are buying

If we substitute our factor model in our trade PnL and take expectations what do we get?

$$1M*E[r^{apple}-rf-\beta*r^{SPY}+\beta*rf)]$$

$$1M*E[\alpha_i+rf+beta*(r^{SPY}-rf)+u_i-rf-\beta*r^{SPY}+\beta*rf)]$$

A pure play portfolio!

$$1M*E[\alpha_i]$$

And what is the portfolio risk?

$$std(1M*(r^{apple}-rf-\beta*r^{SPY}+\beta*rf))=1M*std(u_i)$$





Lets do this!

I will use a regression to get some numbers for us to do this strategy

but for now, just think those as numbers I am giving you and don't think of the regression as giving the right numbers necessarily.


Do example with an asset. Compare the hedged and unhedged returns

Discuss why these are trading strategy returns

Show the positions that Implement this trade

In [16]:
conn = wrds.Connection()
ticker=['MSFT']
df_returns = get_daily_wrds(conn,ticker)
print(df_returns.head())
df_returns.describe()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
[10107]
ticker          MSFT
date                
1986-03-14  0.035714
1986-03-17  0.017241
1986-03-18 -0.025424
1986-03-19 -0.017391
1986-03-20 -0.026549


ticker,MSFT
count,9526.000000
mean,0.001143
std,0.021195
min,-0.301158
25%,-0.009178
50%,0.000360
75%,0.011318
max,0.195652


In [17]:
df_factor = get_factors()
print(df_factor.head())
df_factor.describe()

c:\Users\alan.moreira\Anaconda3\lib\site-packages\pandas_datareader\famafrench.py:114: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = read_csv(StringIO("Date" + src[start:]), **params)


                 RF  Mkt-RF
Date                       
1963-07-01  0.00012 -0.0067
1963-07-02  0.00012  0.0079
1963-07-03  0.00012  0.0063
1963-07-05  0.00012  0.0040
1963-07-08  0.00012 -0.0063


,RF,Mkt-RF
count,15440.000000,15440.000000
mean,0.000173,0.000279
std,0.000125,0.010198
min,0.000000,-0.174400
25%,0.000070,-0.004200
50%,0.000180,0.000500
75%,0.000240,0.005100
max,0.000610,0.113500


In [18]:


df_return, df_factor = df_returns.align(df_factor, join='left', axis=0)

date
1986-03-14    0.035414
1986-03-17    0.016941
1986-03-18   -0.025724
1986-03-19   -0.017691
1986-03-20   -0.026849
                ...   
2023-12-22    0.002574
2023-12-26    0.000004
2023-12-27   -0.001785
2023-12-28    0.003025
2023-12-29    0.001815
Length: 9526, dtype: float64

In [33]:
df_eret=df_return['MSFT']-df_factor['RF']
import statsmodels.api as sm

X = df_factor['Mkt-RF']
X = sm.add_constant(X)  # Adds a constant term to the predictor
y = df_eret

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.414
Model:                            OLS   Adj. R-squared:                  0.414
Method:                 Least Squares   F-statistic:                     6731.
Date:                Sat, 14 Dec 2024   Prob (F-statistic):               0.00
Time:                        18:51:57   Log-Likelihood:                 25744.
No. Observations:                9526   AIC:                        -5.148e+04
Df Residuals:                    9524   BIC:                        -5.147e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0006      0.000      3.690      0.0

In [70]:
alpha=model.params[0]
beta=model.params[1]
var_r=y.var()
var_f=X['Mkt-RF'].var()
var_e=model.resid.var()
mu_f=X['Mkt-RF'].mean()
print(f"Alpha: {alpha}")
print(f"Beta: {beta}")
print(f"Variance of MSFT returns (var_r): {var_r}")
print(f"Variance of market factor (var_f): {var_f}")
print(f"Variance of residuals (var_e): {var_e}")
print(f"Mean of market factor (mu_f): {mu_f}")

Alpha: 0.0006136713970132121
Beta: 1.1885965257521751
Variance of MSFT returns (var_r): 0.00044916491215715146
Variance of market factor (var_f): 0.0001316572593818388
Variance of residuals (var_e): 0.00026316457843989904
Mean of market factor (mu_f): 0.00034711316397228635


C:\Users\alan.moreira\AppData\Local\Temp\ipykernel_6532\2604691240.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha=model.params[0]
C:\Users\alan.moreira\AppData\Local\Temp\ipykernel_6532\2604691240.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta=model.params[1]


Questions
 - What frequency are these parmeters? How to I make it comparable say with the annual risk-free rate?
 - What does the alpha mean? How should you think about it? Do you think is informative about future MSFT alpha? Why? Why not?
 - What about the variance terms? And the covariance terms? 

LEts say we have 1 million dollar per month volatility (i.e. standard deviation) budget

What that means? 

Means that the size of my position times the yearly volatility of my position must be less than 1 million dollars?

> Why you might impose such a cap?

How much o MSFT can I own?

- If I invest one dollar in MSFT I get: $1*\sqrt(var_r*252)$ standard deviation for 1 year
- If I invest one dollar in MSFT I get: $1*\sqrt(var_r*21)$ standard deviation for 1 month

To get 1M per month I invest $\frac{1M}{\sqrt(var_r*21)}$



In [73]:
x=1/(var_r*252)**0.5
x

2.9723285858071415

> What does that mean? How many million dollars I can buy of the stock?

What if the budget was for 1 year? How many would I be able to buy?


1. What is your expected PnL in the end of the year?

    - Here we will subtract you funding costs, which I will assume is also rf so it will be a wash

$$x*(rf+\beta*\mu_f+\alpha-rf)*252$$


1. What is your PnL after your market exposure is subtracted

$$x*(\alpha)*252$$

In [67]:

print(x*(beta*mu_f+alpha)*252)
print(x*(alpha)*252)

0.7686879316063537
0.4596563249799126


Now suppose you focus your edge--i.e. investing in MSFT but hedging the market risk

1. How much of the market do I need to short per dollar invested in MSFT?

you short exactly it's beta

$$x_h=-\beta$$

So your portfolio is

$$x*(stock:1,market:-\beta)$$

2. How many dollars in MSFT can I buy?

$$\frac{1M}{\sqrt(var_e*252)}$$



In [64]:
xe=1/(var_e*252)**0.5
xe

3.8831665045807324

Why $var_e$ is the volatility of the hedge portfolio?

Note that the hedged portfolio return is

$$rh=re-\beta*f=\alpha+\epsilon$$

Note that the OLS regression implies $E[f\epsilon]=0$

The leftover risk is exactly orthogonal to the factor risk

This what makes this the optimal hedge portfolio--> it hedges all the market risk!

So it follows that

$$Var(rh)=Var(\alpha+\epsilon)=Var(\epsilon)$$

1. What is you expected "factor-adjusted PnL"?



In [74]:
xe*(alpha*252)

0.6005130298526444

Some shops pay you for your full PnL but Impose constraints on how much factor risk you can have. For example they might impose a maximum 20% of your total risk in factor risk

Are you better off hedging everything anyways? Or do you want to hedge just enough to be at 20%? 


How do you know how much to hedge? 



Why should you care about your portfolio risk?

- suppose you have 1M per year vol , what is the most money you expect to loose with 5% probability?

First lets assume it is normally distributed.

(we often will do that because make it easier to think, but you should always keep in mind that is an approximation)


$$Prob(r<X)=5\%$$

IT is convenient to transform into a stantard normal so we get a clean formula

$$Prob\left(\frac{r-E[r]}{std(r)}<\frac{X-E[r]}{std(r)}\right)=5\%$$

This is just the CDF of standard formal distribution. Let $F(x)=Prob(z<x)$ where $z\sim N(0,1)$


$$F\left(\frac{X-E[r]}{std(r)}\right)=5\%$$

Then we can use the inverse CDF, which is the quantile function

$$F^{−1}(p)=min\{x∣F(x)≥p\}$$

It gives use the lowest value x that is consistent with the probability being higher then a given value


$$\frac{X-E[r]}{std(r)}=F^{-1}(5\%)$$

Then 

$$X=F^{-1}(5\%)*std(r)+E[r]$$

In [68]:
from scipy.stats import norm
x=0.05
norm.ppf(x)


-1.6448536269514729

First lets do the unhedged. 

In [78]:
prob=0.05
mu=x*(beta*mu_f+alpha)*252
sig=x*(var_r*252)**0.5
norm.ppf(prob)*sig+mu

-0.8761656953451189

How do you interpret this value? 

It means that there is a 5% probability that you will lose more than 800 thousand dollars

Lets do the hedged

In [76]:
prob=0.05
mu=xe*(alpha)*252
sig=xe*(var_e*252)**0.5
norm.ppf(prob)*sig+mu

-1.0443405970988286

Why are you risking a worse loss in the case of the hedged?

Lets suppose 

## Variance Decomposition


  






Factor models are essential to manage risk

A pod shop mananager , a manager in a mutual fund in a large mutual fund company, a hedge fund trader in a large hedge fund,...all of those will typically have tight limits on the TOTAL risk of their portfolio, i.e. $var(r)$ but also on the factor component of their portfolio

But even if you don't, and you are an active manager with freedom of trading, you still want to control quite carefully how much non-essential risks you are taking in your portfolio

The decomposition here with single factor is really easy, but once you understand it, the multi-factor decomposition will be also easy to understand ( even it a little more involved)

1. The total variance of your strategy/asset is

$$Var(r^e)=Var(\alpha +b*f+\epsilon)=Var(b*f)+Var(\epsilon)=b^2Var(f)+var(\epsilon)$$

We can see already the factor part $(b^2Var(f))$ and the idiosyncratic part $var(\epsilon)$

2. The fraction of factor risk in your portfolio is

$$\frac{b^2Var(f)}{Var(r^e)}$$

You factor risk depends

- on your factor beta--this might grow in a market turmoil for example
- on the factor variance--also might grow when markets are choppy

The factor risk-share also depend on the idio vol. During a market turmoil all vols tend to go up, but factor vols tend to go up more
So both your total risk grows as the share of factor risk

In [79]:
beta**2*var_f/var_r

0.4141025460425463

Suppose I want to get this to the 20% limit. What do I do?

Lets say xh is your position on the market, then your portfolio is

$$rp=x(1*re+xh*f)$$

You want the factor piece to be less than vbar=20%

$$Var(rp)=Var(x(1*re+xh*f))=Var(x(1*(\alpha+\beta*f+\epsilon)+xh*f))=Var(x(\beta+xh)*f)+x^2Var(\epsilon)$$

$$Var(rp)=x^2(\beta+xh)^2*Var(f)+x^2Var(\epsilon)$$

so we need


$$\frac{x^2(\beta+xh)^2*Var(f)}{x^2(\beta+xh)^2*Var(f)+x^2Var(\epsilon)}=vbar$$

Note that your size of your overall portfolio cancels out. 

What matters for the share is your hedge per unit of position

$$\frac{(\beta+xh)^2*Var(f)}{(\beta+xh)^2*Var(f)+Var(\epsilon)}=vbar$$

reorganizing


$$(\beta+xh)^2*Var(f)=vbar*((\beta+xh)^2*Var(f)+Var(\epsilon))$$

$$(1-vbar)(\beta+xh)^2*Var(f)=vbar*Var(\epsilon)$$

$$(\beta+xh)^2=\frac{vbar*Var(\epsilon)}{(1-vbar)Var(f)}$$

$$(\beta+xh)=\sqrt{vbar\frac{Var(\epsilon)}{(1-vbar)Var(f)}}$$

$$xh=\sqrt{\frac{vbar*Var(\epsilon)}{(1-vbar)Var(f)}}-\beta$$



What is the hedge portfolio if you want zero factor risk?

In [80]:
vbar=0.2
xh20=((vbar*var_e)/((1-vbar)*var_f))**0.5-beta
xh20

-0.4816910986685352

## Extra material: Factor models means you need to estimate LESS parameters



We will see how we can compute covariance matrixes easily under the assumption that the factor captures most sources of co-movement across stocks

In this case the assumption is very bad...and not even close to holding for most assets...but will allow us to get the intuition

We will then learn how to do it with many factors as is standard in the industry


Here I will use i,j for a individual stock


$$r^{e,i}=\alpha_i+\beta_{i,f}f+\epsilon_{i}$$

- non-factor component $\epsilon_i$ of the asset return of asset $i$. Assumed to uncorrelated between any two assets

Now this is just a regression!

We have that

$$Var(r^{e,i})=\beta_i^2Var(f)+var(\epsilon_{i})$$


This gives us a nice decomposition of risk as we saw above

- the risk component that is driven by the common factor
- the asset specific comment


Now if our assumption holds,i.e. the non-factor risk is uncorrelated across assets

 $$cov(\epsilon_{i,t},\epsilon_{j,t})=0$$

Then 

$$Cov(r_t^{e,i},r_t^{e,j})=Cov(\alpha_i+\beta_{i}f_t+\epsilon_{i,t},\alpha_j+\beta_{j}f_t+\epsilon_{j,t})=\beta_i\beta_jVar(f_t)$$

And the covariance matrix is simply


$$
\left[\begin{array}{cc}
\beta_i^2 & \beta_i\beta_j \\
\beta_i\beta_j &  \beta_j^2 
\end{array}\right]var(f_t)+\left[\begin{array}{cc}
var(\epsilon_{i,t}) & 0 \\
0&  var(\epsilon_{j,t})
\end{array}\right]$$

We see that now for N assets we only need to estimate N factor exposures, N asset specific volatilities , and the factor volatility

>$2N+1$ vs $(N^2-N)/2+N$


That is a huge difference


Suppose we have many assets, all respecting this factor model. Say $R_t^e$ is the big N by 1 vector of excess return $[r_{t}^{e,1},r_{t}^{e,2},...]$ for all these assets

$$R_t^e=Bf_t+U_t$$

where $B$ is also a N by 1 vector of exposures $[\beta_1,\beta_2,...\beta_N]$  and $u_t$ is the N by 1 vector of shocks $[\epsilon_1,\epsilon_2,...]$


Then their variance-covariance matrix is simply


$$Var(R^e_t)=B @ B' Var(f_t)+ Var(U_t)$$

- Armed with the variance covariance of assets we have the variance of any portfolio of these assets (wait!)

 
For the few stocks we looked at above, can you?

- Estimate their unrestricted covariance matrix?
- estimate the single-factor model implied covariance matrix?
- Can you compare the number of parameters you need to estimate in each case?
- Can you discuss if it such model is a good one in this case? 
- 